In [ ]:
pip install selenium

In [ ]:
#importing relevent libraries

from selenium import webdriver 
import time 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd

"""
The "chromedriver", that is mandatory to run selenium library, was installed and stored in the same directory as this python file
And it was also stored at the system PATH.
It doesn't matter what browser you're running the notebook on, as long as you have Goggle Chrome on your Pc.
"""

In [ ]:
'''
After executing this cell let the new YouTube window open and load before proceeding to next cell.
Minimize the other other window and then proceed to execute the next cell.
'''

driver = webdriver.Chrome() 
driver.get("https://www.youtube.com/results?search_query=cakes")

links = []
#this list will contain urls of all the pages we are supposed to visit

df = pd.DataFrame(columns = ['link', 'title', "no_of_views", "uploaded_date", "top_comments"])
#this is the dataframe in which we will store the output

In [ ]:
"""
this function does the following:
1) extract the urls of all videos presently visible on the page and appends them in a list.
2) iterates through that list visiting each page one by one.
3) upon visiting each page, it extracts its date, views, likes and atleast one comment with the help of Explicit Wait 
& Expected Conditions class of selenium, and appends it as a new row in our dataframe.

Note 1): this will take a significant amount of time to finish complete execution, 
and for that reason we are working on only around 20-25 youtube links.
(Although if we want to iterate through more links, a few simple lines of code can be added, and small modifications can be made)

Note 2): it requires having a fast internet connection, so that the pages and comments are generated timely,
else it throws "TimeOutException";

Note 3): it will require constant monitoring of the youtube page running in parallel.
"""

def youtube_scraping():
    
    wait = WebDriverWait(driver, 20)

    user_data = driver.find_elements_by_xpath('//*[@id="video-title"]')
    for i in user_data:
        links.append(i.get_attribute('href'))

    for x in links:
        driver.get(x)
        v_link = x
        v_title = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,"h1.title yt-formatted-string"))).text
        v_views = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "yt-view-count-renderer"))).text
        v_date = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div#date yt-formatted-string"))).text
        v_comments = []
        wait.until(EC.visibility_of_element_located((By.TAG_NAME, "body"))).send_keys(Keys.END)
        for comment in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#content-text"))):
            v_comments.append(comment.text)
        df.loc[len(df)] = [v_link, v_title, v_views, v_date, v_comments]
        
'''
P.S. Not to be a spoil sport, but extracting the number of likes and dislikes on each video was a tad bit difficult
(because they have the same "id" and "class", as you will see if you "inpect element") and hence has been omitted.
Hope to master it in future.
'''

In [ ]:
youtube_scraping() #calling the function

In [ ]:
len(df) #shows us how many web pages we were able to read

In [ ]:
df

In [ ]:
df.to_csv("output.csv", index=False) #stored in the same directory as your notebook